In [1]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los dfFrames

from tqdm import tqdm

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Librería para el acceso a variables y funciones
# -----------------------------------------------------------------------
import sys
sys.path.append("../")
from src import soporte_rentabilidad as sr
from src import soporte_yolo as sy

In [2]:
paths_transformers = ["../transformers/target_encoder.pkl", "../transformers/scaler.pkl", "../transformers/model.pkl"]
df = sr.predecir_alquiler("../data/transformed/final_sale.pkl", paths_transformers)

In [3]:
df.head(1)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",FINCAS RUIZ,876210884,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.86935 41.6973),Distrito Rural,1180.285


In [14]:
df = sy.identificar_urls_habitaciones(df, 'urls_imagenes')

100%|██████████| 365/365 [14:45<00:00,  2.43s/it]


Se han eliminado 93 filas, donde 'url_cocina' o 'url_banio' son None o NaN.


In [15]:
df[df['anunciante']=='ND'].sample(3)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho,url_cocina,url_banio
289,107007692,124000.0,2340.0,piso,True,2,False,53.0,1,1,False,False,False,False,good,"calle de Añón, 24",Apartamento rústico-vintage de un dormitorio e...,ND,669797862,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.87316 41.65038),Casco Histórico,815.53000,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
131,35110384,149000.0,1713.0,piso,True,1,False,87.0,3,1,False,True,True,False,good,"calle Goya, 10","Reforma integral: eléctricidad, tuberías sanit...",ND,625675251,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-1.0289 41.7224),Distrito Rural,891.01619,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
416,106112735,150000.0,1744.0,piso,True,1,False,86.0,3,1,False,True,True,False,good,"paseo de Calanda, 23",Se pone a la venta piso en Paseo Calanda. Un p...,ND,ND,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.90345 41.65206),Delicias,941.57500,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...


In [16]:
df.to_pickle("../data/transformed/final_tags.pkl")